In [1]:
import sys
#notification导表到notification_etl_s3
def install():
    sc.uninstall_package('boto3')
    sc.install_pypi_package("boto3")
install()
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
import boto3
import json
import pytz
import hmac
import pyspark.sql.functions as F
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql import types as T
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
tablemap={"notification_stream_etl":"notification"} 
# {"notification_stream_etl":"notification"} #,
# {"telemarket_stream_etl":"telemarket"} #,
# ,"banda_stream_etl":"1"
tablesql_1=" FROM  (   SELECT    *  , row_number() OVER (PARTITION BY id  ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC  , if(etlindex is null ,0,etlindex)  desc)  row_num FROM  "
tablesql_2=" ORDER BY id  ASC )  a  WHERE a.row_num = 1   AND a.kind <> 'delete' "
# 替换手机号
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None:
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b):
    colum=""
    for index in range(len(b)):
        if(index>0 and index<len(b)-8):   
            colum=colum+setDef(b[index]["data_type"],b[index]["col_name"])+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col):
    if(type[:7] == 'decimal'):
        return "ifnull("+table_col+",0) as "+table_col
    else:
        return table_col
def add_secret_col(dbType,tableNm,col):
    secret_Col=[]
    if colmap[tablemap[databaseName]]!=None and colmap[tablemap[databaseName]].get(tableNm)!=None:
        secret_Col=colmap[tablemap[databaseName]][tableNm]
    sh256Col =col
    hmac_key=colmap["hmac_key"]
#     sql="select hmac_sha256('{hmac_key}','123456789') as sh2".format(hmac_key=hmac_key)
    for i in secret_Col:
        sh256Col=sh256Col+ ", hmac_sha256('{hmac_key}','"+i+"', "+i+") "+ i+"_x"
    return  sh256Col.format(hmac_key=hmac_key)
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
    if tableName=='t_message':
        tablecolum="desc " +databaseName+"."+tableName;
        tableSchema= spark.sql(tablecolum).collect()
        tablepath="s3://rupiahplus-data-warehouse/aliyun/"+tablemap[databaseName]+"/"+row["tableName"]
        spark.catalog.refreshTable(databaseName+"."+tableName)
        secret_sql=add_secret_col(tablemap[databaseName],tableName,getTableColum(tableSchema))+", year"
        sql="select "+secret_sql+ tablesql_1+databaseName+"."+tableName+tablesql_2
        df=spark.sql(sql).drop("etlindex")
        if colmap[tablemap[databaseName]].get(tableName)!=None :
            for i in colmap[tablemap[databaseName]][tableName]:
                df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
        df.drop("col_x").write.mode("overwrite").partitionBy("year").orc(tablepath)
if __name__ == "builtins":
#     注册udf
    spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for databaseName in tablemap:
        print(databaseName)
        databasesql="show tables in "+databaseName
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in tablelist])
            for  future in futures_result[0]:
                print(future.result())

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1620358127926_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



<function hmac_sha256 at 0x7fe2c9a1dd40>
notification_stream_etl
None
None
None
None
None
None
Cannot uninstall requirement boto3, not installed

In [4]:
import sys
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from datetime import datetime,timedelta
import copy
import json
import pytz
import hmac
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql import types as T
spark = SparkSession\
    .builder\
    .appName("Python Demo")\
    .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
    .config("spark.driver.maxResultSize", "4g") \
    .enableHiveSupport()\
    .getOrCreate()
spark.conf.set("hive.exec.dynamic.partition.mode","nonstrict");
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("spark.sql.shuffle.partitions",20)
spark.conf.set("spark.files.overwrite","true")
#     ,Apache Arrow：一个跨平台的在内存中以列式存储的数据层，用来加速大数据分析速度。其可以一次性传入更大块的数据，pyspark中已经有载入该模块，需要打开该设置：
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
#     创建表时自动删除已存在的目录
spark.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
dbmap={
    "notification":{
        "increment_database":"notification_stream_etl",
        "database":"notification_etl_s3",
        "realTime_path":"notification"
    },
#      "telemarket":{
#          "increment_database":"telemarket_stream_etl",
#          "database":"telemarket_etl_s3",
#          "realTime_path":"telemarket"
#     }
}
nowdate=(datetime.now()).strftime('%Y-%m-%d')
yesterday=(datetime.now()+ timedelta(-1)).strftime('%Y-%m-%d')
sqltemp="""with {update_snapshot_table} as (
  select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
         FROM `{increment_database}`.`{tableNm}`
         where date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
      )
    where row_num = 1
  )
select  {select_sql} 
from `{database}`.`{tableNm}` l
left join {update_snapshot_table}
on l.id = {update_snapshot_table}.id
where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete'
union
select {update_snapshot_sql} from (
  select *
  from `{increment_database}`.`{tableNm}`
  where
    date(year || '-' || month || '-' || day) <=date('{nowdate}') and date(year || '-' || month || '-' || day) >= date('{yesterday}')
    and kind = 'insert'
  ) new
left join {update_snapshot_table}
on new.id = {update_snapshot_table}.id where if({update_snapshot_table}.kind is not null,{update_snapshot_table}.kind,'') <> 'delete' """
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()   
hmac_key=colmap["hmac_key"]
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None and value!='':
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b,update_snapshot_table,dbtype,tableNm):
    colum=""
    snapshot_colum=""
    sqlstrtemp="if({update_snapshot_table}.id is null, {colNm}     , {snapshot_colNm})  {ascolNm} ,"
    sqlstr_secret_temp=" if({update_snapshot_table}.id is null, {colNm}     , hmac_sha256('{hmac_key}','{ascolNm}',{snapshot_colNm})) {ascolNm}_x , "
    for index in range(len(b)):
        if(index==1 or (index>2 and index<len(b)-7)):
#             print(index,b[index])
#             在year前面加上加密列
            if(index==len(b)-8)and  colmap.get(dbtype)!=None and colmap[dbtype]!=None and colmap[dbtype].get(tableNm)!=None:
                for secret_col in colmap[dbtype][tableNm]:
                    sqlstr=copy.copy(sqlstr_secret_temp)
                    colum=colum+sqlstr.format(colNm=setDef("string",secret_col,False),snapshot_colNm=setDef("string",secret_col,True),ascolNm=secret_col,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
                    snapshot_colum=snapshot_colum+setsecret_SnapshotDef(secret_col).format(update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)+" ,"
            sqlstr=copy.copy(sqlstrtemp)
            colum=colum+sqlstr.format(colNm=isX(dbtype,tableNm,setDef(b[index]["data_type"],b[index]["col_name"],False)),snapshot_colNm=setDef(b[index]["data_type"],b[index]["col_name"],True),ascolNm=b[index]["col_name"],update_snapshot_table=update_snapshot_table)
            snapshot_colum=snapshot_colum+setSnapshotDef(b[index]["data_type"],b[index]["col_name"]).format(update_snapshot_table=update_snapshot_table)+" ,"
    return colum[0:len(colum)-2],snapshot_colum[0:len(snapshot_colum)-2]
def isX(type,table,column):
    columnList=column.split(".")
    if len(columnList)>0 and  colmap.get(type)!=None and colmap[type].get(table)!=None and columnList[len(columnList)-1] in colmap[type][table]:
        return column+"_x"
    return column
def setDef(type,table_col,is_snapshot):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        if is_snapshot:
            return "ifnull({update_snapshot_table}."+table_col_x+",0)  "
        else:
            return "ifnull(l."+table_col_x+",0) "
    else:
        if is_snapshot:
            return "{update_snapshot_table}."+table_col_x
        else:
            return "l."+table_col_x
def setSnapshotDef(type,table_col):
    table_col_x=copy.copy(table_col)
    if(type[:7] == 'decimal'):
        return "ifnull({update_snapshot_table}."+table_col_x+",0) "+table_col
    else:
        return "{update_snapshot_table}."+table_col_x
def setsecret_SnapshotDef(table_col):
    table_col_x=copy.copy(table_col)
#     print("hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x")
    return "hmac_sha256('{hmac_key}','"+table_col_x+"', {update_snapshot_table}."+table_col_x+") "+ table_col_x+"_x"
#     return "sha2({update_snapshot_table}."+table_col+",256) " +table_col+"_x"
def execute(tablerow,dbtype):
#     print(tablerow,dbtype)
    tableName=tablerow["tableName"]
    if tableName=='t_message':
        update_snapshot_table="update_snapshot_"+dbtype+"_"+tableName
        spark.catalog.refreshTable("`"+dbmap[dbtype]["database"]+"`."+tableName)
        col=spark.sql("desc `"+dbmap[dbtype]["increment_database"]+"`."+tableName).collect()
        #     #真正的列,增量的列
        real_col,snapshot_col=getTableColum(col,update_snapshot_table,dbtype,tableName)
        sql=copy.copy(sqltemp)
        real_sql=sql.format(increment_database=dbmap[dbtype]["increment_database"],tableNm=tableName,nowdate=nowdate,select_sql=real_col.format(update_snapshot_table=update_snapshot_table),update_snapshot_sql=snapshot_col.format(update_snapshot_table=update_snapshot_table),database=dbmap[dbtype]["database"],yesterday=yesterday,update_snapshot_table=update_snapshot_table,hmac_key=hmac_key)
#         df=spark.sql(real_sql).drop("etlindex")
#         if colmap.get(dbtype)!=None and colmap[dbtype].get(tableName)!=None :
#             for i in colmap[dbtype][tableName]:
#                 df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
#         df.drop("col_x").write.mode("overwrite").partitionBy("year").format("orc").saveAsTable("etl_s3_temp."+dbtype+"_"+tableName);
#         tablepath='s3://rupiahplus-data-warehouse/aliyun/'+dbmap[dbtype]["realTime_path"]+'/'+tableName+"/"
#         spark.sql("select  * from  "+"etl_s3_temp."+dbtype+"_"+tableName).write.mode("overwrite").partitionBy("year").orc(tablepath)
        print(real_sql)
if __name__ == "builtins":
    print("start-----",datetime.now())
    for dbtype in dbmap:
        databasesql="show tables in "+dbmap[dbtype]["increment_database"]
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
    #         for row in tablelist:
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(execute, table,dbtype) for table in tablelist])
            for  future in futures_result[0]:
                print(future.result())
                print("end-----",datetime.now())

VBox()

Exception in thread cell_monitor-4:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

start----- 2021-02-26 09:57:25.016937
with update_snapshot_notification_t_message as (
  select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
         FROM `notification_stream_etl`.`t_message`
         where date(year || '-' || month || '-' || day) <=date('2021-02-26') and date(year || '-' || month || '-' || day) >= date('2021-02-25')
      )
    where row_num = 1
  )
select  if(update_snapshot_notification_t_message.id is null, l.etldate     , update_snapshot_notification_t_message.etldate)  etldate ,if(update_snapshot_notification_t_message.id is null, l.id     , update_snapshot_notification_t_message.id)  id ,if(update_snapshot_notification_t_message.id is null, l.product_id     , update_snapshot_notification_t_message.product_id)  product_id ,if(update_snapshot_notification_t_message.id is null, l.ref_id     , update_snapshot_notification_t_mess

In [4]:
df=spark.sql("""
with update_snapshot_notification_t_message as (
  select *
    from (
      SELECT * ,
         row_number() OVER (PARTITION BY id ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC, if(etlindex is NULL, 0, etlindex) desc) row_num
         FROM `notification_stream_etl`.`t_message`
         where date(year || '-' || month || '-' || day) <=date('2021-02-25') and date(year || '-' || month || '-' || day) >= date('2021-02-24')
      )
    where row_num = 1
  )
select update_snapshot_notification_t_message.etldate ,update_snapshot_notification_t_message.id ,update_snapshot_notification_t_message.product_id ,update_snapshot_notification_t_message.ref_id ,update_snapshot_notification_t_message.type ,update_snapshot_notification_t_message.sub_type ,update_snapshot_notification_t_message.channel ,update_snapshot_notification_t_message.send_to ,update_snapshot_notification_t_message.status ,update_snapshot_notification_t_message.title ,update_snapshot_notification_t_message.content ,update_snapshot_notification_t_message.result_description ,update_snapshot_notification_t_message.expect_send_time ,update_snapshot_notification_t_message.actual_send_time ,update_snapshot_notification_t_message.create_time ,update_snapshot_notification_t_message.update_time ,update_snapshot_notification_t_message.avatar ,update_snapshot_notification_t_message.creative ,update_snapshot_notification_t_message.trigger_id ,update_snapshot_notification_t_message.trigger_type_id ,update_snapshot_notification_t_message.params  ,update_snapshot_notification_t_message.year from (
  select *
  from `notification_stream_etl`.`t_message`
  where
    date(year || '-' || month || '-' || day) <=date('2021-02-25') and date(year || '-' || month || '-' || day) >= date('2021-02-24')
    and kind = 'insert'
  ) new
left join update_snapshot_notification_t_message
on new.id = update_snapshot_notification_t_message.id where if(update_snapshot_notification_t_message.kind is not null,update_snapshot_notification_t_message.kind,'') <> 'delete'
and  update_snapshot_notification_t_message.id=48326289
""").show()
# df=df.drop("etlindex")
# df.drop("col_x").write.mode("overwrite").partitionBy("year").format("orc").saveAsTable("etl_s3_temp.notification_t_message")
# spark.sql("select  * from  "+"etl_s3_temp.notification_t_message").show()
# print(real_sql)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+----------+--------------------+----+--------+-------+--------------------+-------+--------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------+--------+----------+---------------+--------------------+----+
|             etldate|      id|product_id|              ref_id|type|sub_type|channel|             send_to| status|   title|             content|  result_description|   expect_send_time|    actual_send_time|         create_time|         update_time|avatar|creative|trigger_id|trigger_type_id|              params|year|
+--------------------+--------+----------+--------------------+----+--------+-------+--------------------+-------+--------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------+--------+----------+---------------+--------------------+----+
|2021-02-25 03:06:...|48326289|         0|a47c

In [ ]:
select  if(update_snapshot_notification_t_message.id is null, l.etldate     , update_snapshot_notification_t_message.etldate)  etldate ,if(update_snapshot_notification_t_message.id is null, l.id     , update_snapshot_notification_t_message.id)  id ,if(update_snapshot_notification_t_message.id is null, l.product_id     , update_snapshot_notification_t_message.product_id)  product_id ,if(update_snapshot_notification_t_message.id is null, l.ref_id     , update_snapshot_notification_t_message.ref_id)  ref_id ,if(update_snapshot_notification_t_message.id is null, l.type     , update_snapshot_notification_t_message.type)  type ,if(update_snapshot_notification_t_message.id is null, l.sub_type     , update_snapshot_notification_t_message.sub_type)  sub_type ,if(update_snapshot_notification_t_message.id is null, l.channel     , update_snapshot_notification_t_message.channel)  channel ,if(update_snapshot_notification_t_message.id is null, l.send_to_x     , update_snapshot_notification_t_message.send_to)  send_to ,if(update_snapshot_notification_t_message.id is null, l.status     , update_snapshot_notification_t_message.status)  status ,if(update_snapshot_notification_t_message.id is null, l.title     , update_snapshot_notification_t_message.title)  title ,if(update_snapshot_notification_t_message.id is null, l.content     , update_snapshot_notification_t_message.content)  content ,if(update_snapshot_notification_t_message.id is null, l.result_description     , update_snapshot_notification_t_message.result_description)  result_description ,if(update_snapshot_notification_t_message.id is null, l.expect_send_time     , update_snapshot_notification_t_message.expect_send_time)  expect_send_time ,if(update_snapshot_notification_t_message.id is null, l.actual_send_time     , update_snapshot_notification_t_message.actual_send_time)  actual_send_time ,if(update_snapshot_notification_t_message.id is null, l.create_time     , update_snapshot_notification_t_message.create_time)  create_time ,if(update_snapshot_notification_t_message.id is null, l.update_time     , update_snapshot_notification_t_message.update_time)  update_time ,if(update_snapshot_notification_t_message.id is null, l.avatar     , update_snapshot_notification_t_message.avatar)  avatar ,if(update_snapshot_notification_t_message.id is null, l.creative     , update_snapshot_notification_t_message.creative)  creative ,if(update_snapshot_notification_t_message.id is null, l.trigger_id     , update_snapshot_notification_t_message.trigger_id)  trigger_id ,if(update_snapshot_notification_t_message.id is null, l.trigger_type_id     , update_snapshot_notification_t_message.trigger_type_id)  trigger_type_id ,if(update_snapshot_notification_t_message.id is null, l.params     , update_snapshot_notification_t_message.params)  params , if(update_snapshot_notification_t_message.id is null, l.year     , update_snapshot_notification_t_message.year)  year 
from `notification_etl_s3`.`t_message` l
left join update_snapshot_notification_t_message
on l.id = update_snapshot_notification_t_message.id
where if(update_snapshot_notification_t_message.kind is not null,update_snapshot_notification_t_message.kind,'') <> 'delete'
and l.id<>48326289
union